In [271]:
function filter_jl_files(dir_list::Vector{String})
    jl_files = filter(x -> endswith(x, ".jl"), dir_list)
    non_jl_files = setdiff(dir_list, jl_files)
    return jl_files, non_jl_files
end

filter_jl_files (generic function with 1 method)

In [272]:
function add_filepaths!(file_dict::Dict{String, String}, dir_path::String, dir_list::Vector{String})
    for file in dir_list
        file_dict[file] = joinpath(dir_path, file)
    end
end

add_filepaths! (generic function with 1 method)

In [273]:
function find_jl_files(dir_path::String)
    file_fict = Dict{String, String}()
    jl_files, non_jl_files = filter_jl_files(readdir(dir_path))
    add_filepaths(file_dict, dir_path, jl_files)
    new_dirs = filter(x -> isdir(joinpath(dir_path, x)), non_jl_files)
    for new_dir in new_dirs
        merge!(file_dict, find_jl_files(joinpath(dir_path, new_dir)))
    end
    return file_dict
end

find_jl_files (generic function with 1 method)

In [274]:
# Get all Julia files in the src directory
scr_path = joinpath(pwd(), "src")
file_dict = find_jl_files(scr_path)

Dict{String, String} with 145 entries:
  "h2_production_commit.jl"          => "d:\\github_copies\\DOLPHYN-dev\\src\\H…
  "write_reserve_margin_revenue.jl"  => "d:\\github_copies\\DOLPHYN-dev\\src\\G…
  "load_co2_cap.jl"                  => "d:\\github_copies\\DOLPHYN-dev\\src\\G…
  "flexible_demand.jl"               => "d:\\github_copies\\DOLPHYN-dev\\src\\G…
  "investment_discharge.jl"          => "d:\\github_copies\\DOLPHYN-dev\\src\\G…
  "write_time_weights.jl"            => "d:\\github_copies\\DOLPHYN-dev\\src\\G…
  "write_reg.jl"                     => "d:\\github_copies\\DOLPHYN-dev\\src\\G…
  "load_inputs.jl"                   => "d:\\github_copies\\DOLPHYN-dev\\src\\G…
  "emissions_power.jl"               => "d:\\github_copies\\DOLPHYN-dev\\src\\G…
  "load_period_map.jl"               => "d:\\github_copies\\DOLPHYN-dev\\src\\G…
  "h2_pipeline.jl"                   => "d:\\github_copies\\DOLPHYN-dev\\src\\H…
  "load_h2_g2p_variability.jl"       => "d:\\github_copies\\DOLPHYN-de

In [275]:
# For each file, find

In [276]:
l = "@variable(EP, vH2GenCOMMIT[k in H2_GEN_COMMIT, t=1:T] >= 0)"
a = split(l, "@variable(")
println(a)
b = split(a[2], ",")
println(b)
if contains(b[2], "[")
    c = split(b[2], "[")
end
println(c)
d = strip(c[1])
println(d)

SubString{String}["", "EP, vH2GenCOMMIT[k in H2_GEN_COMMIT, t=1:T] >= 0)"]
SubString{String}["EP", " vH2GenCOMMIT[k in H2_GEN_COMMIT", " t=1:T] >= 0)"]
SubString{String}[" vH2GenCOMMIT", "k in H2_GEN_COMMIT"]
vH2GenCOMMIT


In [277]:
# l = "@expression(EP, eTotalH2GenCStart, sum(eTotalH2GenCStartT[t] for t=1:T))"
l = "@expression(EP, eTotalH2GenCStartT[t=1:T], sum(eH2GenCStart[k,t] for k in H2_GEN_COMMIT))"
a = split(l, "@expression(")
println(a)
b = split(a[2], ",")
println(b)
if contains(b[2], "[")
    c = split(b[2], "[")
    d = strip(c[1])
else
    d = b[2]
    strip(d)
end
println(d)

SubString{String}["", "EP, eTotalH2GenCStartT[t=1:T], sum(eH2GenCStart[k,t] for k in H2_GEN_COMMIT))"]


SubString{String}["EP", " eTotalH2GenCStartT[t=1:T]", " sum(eH2GenCStart[k", "t] for k in H2_GEN_COMMIT))"]
eTotalH2GenCStartT


In [278]:
function get_var_name(line::String)
    x = split(line, "@variable(")
    x = split(x[2], ",")
    if contains(x[2], "[")
        x = split(x[2], "[")
        return strip(x[1])
    else
        return strip(x[2])
    end
end

get_var_name (generic function with 1 method)

In [279]:
function get_exp_name(line::String)
    x = split(line, "@expression(")
    x = split(x[2], ",")
    if contains(x[2], "[")
        x = split(x[2], "[")
        return strip(x[1])
    else
        return strip(x[2])
    end
end

get_exp_name (generic function with 1 method)

In [280]:
function filter_exp_var!(exp_and_vars::Dict{String, Dict{String, String}}, line::String)
    if contains(line, "@variable")
        var_name = get_var_name(line)
        exp_and_vars["var"][var_name] = file_name
    elseif contains(line, "@expression")
        exp_name = get_exp_name(line)
        exp_and_vars["exp"][exp_name] = file_name
    end
end

filter_exp_var! (generic function with 1 method)

In [281]:
function find_exp_var_origins_old(file_dict::Dict{String, String})
    exp_and_vars = Dict{String, Dict{String, String}}("var" => Dict{String, String}(), "exp" => Dict{String, String}())
    for (file_name, file_path) in file_dict
        println(file_name)
        open(file_path) do file
            for line in eachline(file)
                filter_exp_var!(exp_and_vars, line)
            end
        end
    end
    return exp_and_vars
end

find_exp_var_origins_old(file_dict)
exp_and_vars["var"]

find_exp_var_origins (generic function with 1 method)

In [ ]:
function clean_file(dir_path::String)
    open(dir_path, "r") do file
        output_lines = ""
        for line in eachline(file)
            if !startswith(line, "#") && line != ""
                output_lines = string(output_lines, strip(line))
            end
        end
        return output_lines
    end
end

In [282]:
function find_exp_var_origins(file_dict::Dict{String, String})
    exp_and_vars = Dict{String, Dict{String, String}}("var" => Dict{String, String}(), "exp" => Dict{String, String}())
    for (file_name, file_path) in file_dict
        println(file_name)
        open(file_path) do file
            file_string = clean_file(dir_path)
            for line in eachline(file)
                filter_exp_var!(exp_and_vars, line)
            end
        end
    end
    return exp_and_vars
end

file_name = "h2_pipeline.jl"
dir_path = file_dict[file_name]
# file_string = Vector{String}
file_string = clean_file(dir_path)

find_exp_var_origins(file_dict)
exp_and_vars["var"]

h2_production_commit.jl
write_reserve_margin_revenue.jl
load_co2_cap.jl
flexible_demand.jl
investment_discharge.jl
write_time_weights.jl
write_reg.jl
load_inputs.jl
emissions_power.jl
load_period_map.jl
h2_pipeline.jl


BoundsError: BoundsError: attempt to access 1-element Vector{SubString{String}} at index [2]

clean_file (generic function with 2 methods)

In [285]:
open(dir_path, "r") do file
    output_lines = ""
    for line in eachline(file)
        if !startswith(line, "#") && line != ""
            # push!(output_lines, line)
            output_lines = string(output_lines, strip(line))
        end
    end
end

"\"\"\"DOLPHYN: Decision Optimization for Low-carbon Power and Hydrogen NetworksCopyright (C) 2022,  Massachusetts Institute of TechnologyThis program is free software; you can redistribute it and/or modifyit under the terms of the GNU General Public License as published by" ⋯ 10351 bytes ⋯ "(EP,begin[p in 1:H2_P, t in INTERIOR_SUBPERIODS],vH2PipeLevel[p, t] ==vH2PipeLevel[p, t-1] - eH2PipeFlow_net[p, t, -1] - eH2PipeFlow_net[p, t, 1]end)@constraints(EP, begin[p in 1:H2_P], vH2NPipe[p] <= inputs[\"pH2_Pipe_No_Max\"][p]end)return EPend # end H2Pipeline module"

In [287]:
a = Dict{String, String}()
q = file_string
w = split(q, "@variable")
for i in 2:length(w)
    x = w[i]
    # println(x)
    x = split(x, ",")
    # println(x)
    if contains(x[2], "[")
        x = split(x[2], "[")
        # println(x)
        a[strip(x[1])] = file_name
    else
        a[strip(x[2])] = file_name
    end
end

In [291]:
file_name = "h2_pipeline.jl"
dir_path = file_dict[file_name]
a = Dict{String, String}()
q = read(dir_path, String)
w = split(q, "@variable")
for i in 2:length(w)
    x = w[i]
    # println(x)
    x = split(x, ",")
    # println(x)
    if contains(x[2], "[")
        x = split(x[2], "[")
        # println(x)
        a[strip(x[1])] = file_name
    else
        a[strip(x[2])] = file_name
    end
end

file_name = "h2_pipeline.jl"
dir_path = file_dict[file_name]
b = Dict{String, String}()
q = read(dir_path, String)
w = split(q, "@expression")
for i in 2:length(w)
    x = w[i]
    # println(x)
    x = split(x, ",")
    # println(x)
    if contains(x[2], "[")
        x = split(x[2], "[")
        # println(x)
        b[strip(x[1])] = file_name
    else
        b[strip(x[2])] = file_name
    end
end


In [292]:
a

Dict{String, String} with 4 entries:
  "vH2NPipe"        => "h2_pipeline.jl"
  "vH2PipeLevel"    => "h2_pipeline.jl"
  "vH2PipeFlow_pos" => "h2_pipeline.jl"
  "vH2PipeFlow_neg" => "h2_pipeline.jl"

In [293]:
b

Dict{String, String} with 7 entries:
  "ePipeZoneDemand"                => "h2_pipeline.jl"
  "eH2PipeFlow_net"                => "h2_pipeline.jl"
  "eCH2Pipe"                       => "h2_pipeline.jl"
  "eH2PowerConsumptionByPipe"      => "h2_pipeline.jl"
  "eH2NPipeNew"                    => "h2_pipeline.jl"
  "eCH2CompPipe"                   => "h2_pipeline.jl"
  "ePowerBalanceH2PipeCompression" => "h2_pipeline.jl"

In [294]:
exp_and_vars = Dict{String, Dict{String, String}}("exp" => Dict{String, String}(), "var" => Dict{String, String}())
file_name = "h2_pipeline.jl"
open(file_dict[file_name], "r") do f
    for line in eachline(f)
        println(line)
        filter_exp_var!(exp_and_vars, line)
    end
end

"""
DOLPHYN: Decision Optimization for Low-carbon Power and Hydrogen Networks
Copyright (C) 2022,  Massachusetts Institute of Technology
This program is free software; you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation; either version 2 of the License, or
(at your option) any later version.
This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.
A complete copy of the GNU General Public License v2 (GPLv2) is available
in LICENSE.txt.  Users uncompressing this from an archive may not have
received this license file.  If not, see <http://www.gnu.org/licenses/>.
"""

@doc raw"""
    h2_pipeline(EP::Model, inputs::Dict, setup::Dict)

This function includes the variables, expressions and objective funtion of hydrogen pipeline.

This

BoundsError: BoundsError: attempt to access 1-element Vector{SubString{String}} at index [2]

In [295]:
exp_and_vars["var"]

Dict{String, String} with 4 entries:
  "vH2NPipe"        => "h2_pipeline.jl"
  "vH2PipeLevel"    => "h2_pipeline.jl"
  "vH2PipeFlow_pos" => "h2_pipeline.jl"
  "vH2PipeFlow_neg" => "h2_pipeline.jl"

In [296]:
exp_and_vars["exp"]

Dict{String, String} with 1 entry:
  "eH2NPipeNew" => "h2_pipeline.jl"

In [297]:
ep_vars = Dict{String, String}()
file_name = "h2_production_commit.jl"
open(file_dict[file_name], "r") do f
    for line in eachline(f)
        # println(line)
        if contains(line, "@variable")
            var_name = get_var_name(line)
            ep_vars[var_name] = file_name
        end
    end
end

In [298]:
ep_vars

Dict{String, String} with 3 entries:
  "vH2GenCOMMIT" => "h2_production_commit.jl"
  "vH2GenShut"   => "h2_production_commit.jl"
  "vH2GenStart"  => "h2_production_commit.jl"